# FM

FM: Feature Machine

- FM解决的问题：大规模稀疏数据下的特征组合问题。


1. 为什么要特征组合：

实践中通过观察大量的样本数据可以发现，某些特征经过关联之后，与label之间的相关性就会提高。例如“USA”与“THANKSGIVING”, "CHINA"与“Chinese New Year"这样的关联特征，对用户的点击有着正向的影响。

2. 如何组合：

数学模型上表达特征$x_i$与$x_j$组合用$x_ix_j$表示，即所有的多项式模型， 通常情况下只考虑二阶多项式模型，也就是特征两两组合的问题。二阶多项式的模型如下：

$$y = w_0+\sum_{i=1}^n w_ix_i+\sum_{i=1}^{n-1}{\sum_{j=i+1}^n{w_{ij}x_ix_j}}$$

其中n代表样本的特征数量，这里的特征是离散化后的特征，$x_i$是第i个特征的值，$w_0,w_i,W_{ij}$是模型参数。从公式来看，模型前半部分就是普通的LR线性组合，后半部分的交叉项就是特征的组合。

3. 如何求解二次项参数 $w_{ij}$

上面模型有个潜在问题：它对组合特征建模，泛化能力比较弱，尤其是在大规模稀疏特征存在的场景下，这个毛病尤其突出，比如CTR预估和推荐排序。

4. FM模型的提出

$$FM: \hat{y}(x)=w_0+\sum_{i=1}^n{w_ix_i}+\sum_{i=1}^n{\sum_{j=i+1}^n{<v_i,v_j>x_ix_j}}$$

$$<v_i,v_j>=\sum_{f=1}^k{v_{i,j} \cdot v_{j,v}}$$

- FM模型也直接引入任意两个特征的二阶特征组合，和SVM模型最大的不同，在于特征组合的权重的计算方法。
- FM对于每个特征，学习一个大小为k的以为向量，于是，两个特征$x_i$和$x_j$的特征组合的权重值，通过特征对应的向量$v_i$和$v_j$的內积$<v_i,v_j>$来表示。
- 这本质上是对特征进行embedding化表征，和目前非常常见的各种实体embedding 本质思想一致的。

5. 为什么说FM的这种特征embedding模型，在大规模稀疏特征引用环境下比较好用？为什么它的泛化能力强呢？

比如，在训练数据里两个特征并未同时在训练实例里见到过，意味着$x_i and x_j$ 一起出现过的次数为0，如果是SVM的模型，是无法学会这个特征组合的权重的。但是因为FM是学习单个特征的embedding，并不依赖于某个特征组合是否出现过，所以只要特征$x_i$和其他任意特征组合没有看到过，但是在预测的时候，如果看到这个新的特征组合，因为$x_i$和$x_j$都能学会自己对应的embedding，所以可以通过內积算出这个新特征组合的权重。这是为何说FM模型泛化能力强的根本原因。

6. 改写FM公式

$$\sum_{i=1}^n{\sum_{j=i+1}^n{<v_i,v_j>x_ix_j}}$$
$$=\frac{1}{2}\sum_{i=1}^n{\sum_{j=1}^n{<v_i,v_j>x_ix_j}}-\frac{1}{2}\sum_{i=1}^n{<v_i,v_j>x_ix_i}$$
$$=\frac{1}{2}(\sum_{i=1}^n{\sum_{j=1}^n{\sum_{f=1}^k{v_{i,f}v_{j,f}x_ix_j}}} - \sum_{i=1}^n{\sum_{f=1}^k{v_{i,f}v_{j,f}x_ix_i}})$$
$$=\frac{1}{2}\sum_{f=1}^k{((\sum_{i=1}^n{v_{i,f}x_i})(\sum_{j=1}^n{v_{j,f}x_j})-\sum_{i=1}^n{v_{i,f}^2x_i^2)}}$$
$$=\frac{1}{2}\sum_{f=1}^k{((\sum_{i=1}^n{v_{i,f}x_i})^2 - \sum_{i=1}^n{v_{i,f}^2x_i^2})}$$

# FFM 

FFM：Field-aware factorization machines

- 相比于FM模型，FFM引入了field的概念，特征可以被归类到field中。

https://www.biaodianfu.com/ctr-fm-ffm-deepfm.html

1. 一个例子：ESPN、Vogue和NBC同属于出版商，Nike、Gucci和Adidas属于fieId广告商。

| Clicked | Publisher(P) | Advertiser(A) | Gender(G) |
|---------|--------------|---------------|-----------|
| Yes     | ESPN         | Nike          | Male      |


在FM模型中，FM的信息包含：$w_{ESPN}\cdot w_{Nike}+w_{ESPN}\cdot w_{Male}+w_{Nike}\cdot w_{Male}$, ESPN会和Nike与Male的权重进行计算，但Nike和Male属于不同的field，所以他们的潜在影响可能会不同。

FFM加入field的概念，对于刚才的例子来说，二次项变为：
$$w_{ESPN,A}\cdot w_{Nike,P}+w_{ESPN,G}\cdot w_{Male,P}+w_{Nike,G}\cdot w_{Male,A}$$

- 对于特征组合（EPSN，Nike）来说，其隐变量采用的是$w_{EPSN,A}$和$w_{Nike,P}$，对于$w_{EPSN,A}$这是因为Nike属于广告主（Advertiser）的field，而第二项$w_{Nike,P}$则是EPSN是广告商（Publisher）的field。
- 对于特征组合（EPSN，Male）来说，$w_{EPSN,G}$是因为Male是用户性别（Gender）的field，而第二项$w_{Male,P}$是因为EPSN是广告商（Publisher）的field。

2. 因此，FFM的数学公式表示为：

$$y(x)=w_0+\sum_{i=1}^d{w_ix_i}+\sum_{i+1}^d{\sum_{j=i+1}^d{(w_{i,f_j}\cdot w_{j,f_i})x_ix_j}}$$

$f_i$和$f_j$分别代表第i个特征和第j个特征所属的field。若field有f个，隐向量的长度为k，则二次项的系数共有$dfk$个，远多于FM模型的$dk$个。此外，隐向量和field相关，并不能像FM模型一样将二次项简化，计算的复杂度是$d^2k$。

通常情况下，每个隐向量只需要学习特定的field的表示，所以有$k_{FFM}<<k_{FM}$